# Getting the coordinates from the cities
Getting the coordinates from [this online file](https://docs.google.com/spreadsheets/d/1FETP5mYMquMH6K1v8O_K6IQ-yLibroEBJPcdT6lLYFw/edit#gid=359373238)
I can streamline this, maybe when I feel like it one time. 

In [1]:
import pandas as pd 
import numpy as np 
from OSMPythonTools.nominatim import Nominatim

In [2]:
#NOTE: this not secure, but that is not important for when we process only this data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [10]:
# credits to https://gis.stackexchange.com/questions/54405/is-there-a-way-to-populate-a-spreadsheet-of-city-names-with-their-latitudes-and

# Converts a iterable of string City names to a list of lat lon stings
# Example: Input - "New York, NY, USA"
#          Output = "40.7127281,-74.0060152"
def QueryLocations(city_names):
    nominatim = Nominatim()
    lats, lons = [], [] 
    for name in city_names:
        print(name)
        city_json = nominatim.query(name).toJSON()[0]

        lats.append(city_json["lat"])
        lons.append(city_json["lon"])
        
    return lats, lons

In [11]:
df = pd.read_csv("data/European AI safety cities - ENAIS locations.csv")

In [16]:
%%capture 
#capture because it produces much unnecessary output
lats, lons = QueryLocations(df['City'])

In [17]:
# have to use these specific column names for https://maps.co/gis/
df['lat'] = lats
df['lng'] = lons

In [18]:
# manually fix some coords
df.loc[df['City'] == "Cracow", 'lat'] = 50.0647
df.loc[df['City'] == "Cracow", 'lng'] = 19.9450

df.loc[df['City'] == "Belgrade", 'lat'] = 44.8125
df.loc[df['City'] == "Belgrade", 'lng'] = 20.4612


In [19]:
# put all info to show on website in description column
df = df.fillna('')
df[['Unnamed: 2', 'Description', 'Media link']].astype("string")

df['Unnamed: 2'] = df['Unnamed: 2'] + "\n\n" + df['Description'] + "\n\n" + df['Media link']

df = df.rename(columns={"Unnamed: 2": "description"})

df.head()

,City,Name,description,Description,Media link,Tags,Hub,lat,lng
0,Cracow,Cracow,We are about to start regular meetings in Febr...,,,,0,50.0647,19.945
1,Aarhus,Aarhus,Aarhus University has an AI safety interested ...,mailto:zaki@apartresearch.com,https://youtu.be/Nx5gIbTsHJM,ENAIS,0,56.1496278,10.2134046
2,Oxford,AI Safety Hub,AI Safety Hub aims to support individuals who ...,,,,1,51.7520131,-1.2578499
3,Groningen,AI Safety Initiative Groningen,We have a lot of AI research and education goi...,aisig.org,mailto:info@aisig.org,ENAIS,1,53.2190652,6.5680077
4,Amsterdam,Amsterdam,The University of Amsterdam produces excellent...,,,ENAIS,1,52.3730796,4.8924534


In [20]:
# check whether hub is missing
if len(np.unique(df['Hub'])) != 2:
    print("CHECK HUB COLUMN, CITIES MIGHT BE MISSING")

In [21]:
df.loc[df['Hub']==0].to_csv("data/cities_with_coords.csv")
df.loc[df['Hub']==1].to_csv("data/hubs_with_coords.csv")